# Pre-setup

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.4.0-bin-hadoop3'

In [4]:
!pip install -q findspark

import findspark
findspark.init()
findspark.find()

'/content/spark-3.4.0-bin-hadoop3'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
%cd /content/drive/MyDrive/Notebooks/pyspark_practice/frequent_pattern_mining

/content/drive/MyDrive/Notebooks/pyspark_practice/frequent_pattern_mining


# Steps

In [7]:
# create pyspark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FPM").getOrCreate()
spark

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.fpm import FPGrowth
import pyspark.sql.functions as f
from itertools import permutations

## 1. Preprocess

- Đọc tập tin **orders.csv** chứa các giao dịch đã thực hiện. Mỗi giao dịch được thể hiện bằng một hay nhiều dòng, thông tin trên mỗi dòng bao gồm: mã giao dịch (order_id), mã sản phẩm (product_id), thứ tự bỏ vào giỏ (add_to_cart_order), và sản phẩm này có được mua lại hay không (recorded). Ta chỉ quan tâm 2 trường đầu tiên. Sau đó ta lưu lại vào Spark DataFrame `order_df`.

In [10]:
order_df = spark.read.csv('orders.csv', header=True, inferSchema=True)

order_df.printSchema()
order_df.show(5)

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
+--------+----------+-----------------+---------+
only showing top 5 rows



- Đọc tập tin **products.csv** chứa các sản phẩm bày bán trong cửa hàng. Mỗi dòng trình bày thông tin của một sản phẩm, bao gồm: mã sản phẩm (product_id), tên sản phẩm (product_name), và thông tin khác (aisle_id và department_id). Ta chỉ quan tâm 2 trường đầu tiên. Sau đó lưu lại vào Spark DataFrame `product_df`.

In [11]:
product_df = spark.read.csv('products.csv', header=True, inferSchema=True)

product_df.printSchema()
product_df.show(5)

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
+----------+--------------------+--------+-------------+
only showing top 5 rows



- Đưa dữ liệu về một DataFrame gồm 2 cột bao gồm: mã giao dịch `id` và danh sách các sản phẩm `products` bằng cách:
 - B1: INNER JOIN(order_df, product_df) với điều kiện có cùng mã sản phẩm `product_id`.
 - B2: Gom nhóm theo mã giao dịch `order_id` để tạo ra cột mới `products` là một mảng chứa tên sản phẩm. Đổi tên cột mã giao dịch thành `id`. Sau đó lưu lại vào `df`.

In [25]:
# hint: để gom all values cột product_name thành 1 mảng dưới cùng 1 order_id, dùng f.collect_list
df = order_df.join(product_df, order_df['product_id'] == product_df['product_id'], 'inner')\
             .select(['order_id', 'product_name'])\
             .groupby('order_id')\
             .agg(f.collect_list('product_name').alias('products'))\
             .sort('order_id')\
             .withColumnRenamed('order_id', 'id')
df.show(5)            

+---+--------------------+
| id|            products|
+---+--------------------+
|  1|[Bulgarian Yogurt...|
| 36|[Grated Pecorino ...|
| 38|[Shelled Pistachi...|
| 96|[Roasted Turkey, ...|
| 98|[Natural Spring W...|
+---+--------------------+
only showing top 5 rows



## 2. Áp dụng giải thuật khai thác mẫu phổ biến và luật kết hợp 

- Mô hình sử dụng: FPGrowth trong gói pyspark.ml.fpm
- Tiếp cận: chạy thử nghiệm mô hình với 4 ngưỡng support & confidence khác nhau. Với từng mô hình, hiển thị:
 - DataFrame `pattern_df` thể hiện tần số xuất hiện của từng mẫu phổ biến tìm được, gồm 2 cột: tập hạng mục `items` & tần số xuất hiện của từng tập hạng mục `freq`
 - DataFrame `rule_df` thể hiện số luật kết hợp cùng các chỉ số `confidence`, `lift`, `support` của từng luật. Bênh cạnh đó, còn có 2 cột `antecedent` & `consequent` thể hiện vế trái và vế phải của luật (X $\to$ y) 
 - Số lượng mẫu phổ biến và số lượng luật kết hợp
---

**Thử nghiệm 1:** `minSupport = 0.001, minConfidence = 0.4` &#8594; Thời gian xử lý: 1m24s

In [31]:
fpGrowth_1 = FPGrowth(itemsCol = 'products', minSupport = 0.001, minConfidence = 0.4)
model_1 = fpGrowth_1.fit(df)

pattern_df_1 = model_1.freqItemsets
rule_df_1 = model_1.associationRules

pattern_df_1.show(5, False) # False: don't truncate the row
rule_df_1.show(5)
print('Số lượng mẫu phổ biến: ', pattern_df_1.count())
print('Số lượng luật kết hợp: ', rule_df_1.count())

+------------------------------------------------------+----+
|items                                                 |freq|
+------------------------------------------------------+----+
|[White Cheddar Popcorn]                               |370 |
|[Total 0% Nonfat Greek Yogurt]                        |993 |
|[Total 0% Nonfat Greek Yogurt, Organic Avocado]       |137 |
|[Total 0% Nonfat Greek Yogurt, Bag of Organic Bananas]|157 |
|[Total 0% Nonfat Greek Yogurt, Organic Baby Spinach]  |143 |
+------------------------------------------------------+----+
only showing top 5 rows

+--------------------+--------------------+-------------------+-----------------+--------------------+
|          antecedent|          consequent|         confidence|             lift|             support|
+--------------------+--------------------+-------------------+-----------------+--------------------+
|[Apple Honeycrisp...|[Bag of Organic B...| 0.4307692307692308|3.651214470284238|0.001066999977135...|
|[Org

**Thực nghiệm 2:** `minSupport = 0.005, minConfidence = 0.4` &#8594; Thời gian xử lý: 30s

In [32]:
fpGrowth_2 = FPGrowth(itemsCol = 'products', minSupport = 0.005, minConfidence = 0.4)
model_2 = fpGrowth_2.fit(df)

pattern_df_2 = model_2.freqItemsets
rule_df_2 = model_2.associationRules

pattern_df_2.show(5, False)
rule_df_2.show(5)
print('Số lượng mẫu phổ biến: ', pattern_df_2.count())
print('Số lượng luật kết hợp: ', rule_df_2.count())

+----------------------------------+----+
|items                             |freq|
+----------------------------------+----+
|[Total 0% Nonfat Greek Yogurt]    |993 |
|[Organic Large Green Asparagus]   |730 |
|[Green Onions]                    |1445|
|[Organic Tomato Basil Pasta Sauce]|772 |
|[Organic Egg Whites]              |834 |
+----------------------------------+----+
only showing top 5 rows

+--------------------+--------------------+------------------+-----------------+--------------------+
|          antecedent|          consequent|        confidence|             lift|             support|
+--------------------+--------------------+------------------+-----------------+--------------------+
|[Organic Hass Avo...|[Bag of Organic B...|0.4613385315139701|3.910320890272384|0.005411214169759696|
+--------------------+--------------------+------------------+-----------------+--------------------+

Số lượng mẫu phổ biến:  364
Số lượng luật kết hợp:  1


**Thực nghiệm 3:** `minSupport = 0.005, minConfidence = 0.5` &#8594; Thời gian xử lý: 28s

In [33]:
fpGrowth_3 = FPGrowth(itemsCol = 'products', minSupport = 0.005, minConfidence = 0.5)
model_3 = fpGrowth_3.fit(df)

pattern_df_3 = model_3.freqItemsets
rule_df_3 = model_3.associationRules

pattern_df_3.show(5, False)
rule_df_3.show(5)
print('Số lượng mẫu phổ biến: ', pattern_df_3.count())
print('Số lượng luật kết hợp: ', rule_df_3.count())

+----------------------------------+----+
|items                             |freq|
+----------------------------------+----+
|[Total 0% Nonfat Greek Yogurt]    |993 |
|[Organic Large Green Asparagus]   |730 |
|[Green Onions]                    |1445|
|[Organic Tomato Basil Pasta Sauce]|772 |
|[Organic Egg Whites]              |834 |
+----------------------------------+----+
only showing top 5 rows

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+

Số lượng mẫu phổ biến:  364
Số lượng luật kết hợp:  0


**Thực nghiệm 4:** `minSupport = 0.001, minConfidence = 0.5` &#8594; Thời gian xử lý: 1m14s

In [34]:
fpGrowth_4 = FPGrowth(itemsCol = 'products', minSupport = 0.001, minConfidence = 0.5)
model_4 = fpGrowth_4.fit(df)

pattern_df_4 = model_4.freqItemsets
rule_df_4 = model_4.associationRules

pattern_df_4.show(5, False)
rule_df_4.show(5)
print('Số lượng mẫu phổ biến: ', pattern_df_4.count())
print('Số lượng luật kết hợp: ', rule_df_4.count())

+------------------------------------------------------+----+
|items                                                 |freq|
+------------------------------------------------------+----+
|[White Cheddar Popcorn]                               |370 |
|[Total 0% Nonfat Greek Yogurt]                        |993 |
|[Total 0% Nonfat Greek Yogurt, Organic Avocado]       |137 |
|[Total 0% Nonfat Greek Yogurt, Bag of Organic Bananas]|157 |
|[Total 0% Nonfat Greek Yogurt, Organic Baby Spinach]  |143 |
+------------------------------------------------------+----+
only showing top 5 rows

+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|              lift|             support|
+--------------------+--------------------+------------------+------------------+--------------------+
|[Organic Whole St...|[Bag of Organic B...|0.5314685314685315| 4.504745125675359|0.001158457118033...|
|[Org

## Cải thiện giải thuật FP-Growth

**Vấn đề:** Ở cả 4 thực nghiệm, trong các luật được tìm thấy ở `rule_df`, các mệnh đề hệ quả (`consequent`) chỉ có duy nhất một giá trị ([doc](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html) cũng nói về điều này), do đó số lượng luật kết hợp sử dụng thư viện MLlib FBGrowth sẽ ít hơn số lượng luật kết hợp được tìm thấy khi sử dụng các giải thuật khai thác tập phổ biến.

**Giải pháp:** xây dựng hàm *association_rules()* để thực hiện tìm các luật kết hợp của các mẫu phổ biến đã khai thác, nhận đầu vào là dataframe ban đầu `df`, dataframe khai thác các mẫu phổ biến và tần số xuất hiện của nó `pattern_df` và 2 giá trị `minSupport` và `minConfidence`. Sau đó thực hiện lần lượt các bước sau:
- B1: Trong `pattern_df`, ta chỉ lấy các tập phổ biến có từ 2 phần tử trở lên, sau đó lưu vào dataframe `new_pattern_df`.
- B2: Tìm các hoán vị cách chia thành 2 tập nguyên nhân (antecedent) và hệ quả (consequent) của từng phần tử trong `new_pattern_df`, sau đó lưu các hoán vị này vào dataframe `new_df` mà vẫn giữ nguyên tần số xuất hiện của nó (frequency).

    *Ví dụ: Ta có tập phổ biến a = [2, 3, 5] sẽ có các hoán vị nguyên nhân-hệ quả sau:*
    - [2], [3, 5]
    - [2, 3], [5]
    - [2, 5], [3]
    - [3], [2, 5]
    - [3, 2], [5]
    - [3, 5], [2]
    - [5], [2, 3]
    - [5, 2], [3]
    - [5, 3], [2]


- B3: Từ dataframe `new_df`, ta sẽ tính các chỉ số `confidence`, `lift`, `support` của từng phần tử, sau đó lọc theo điều kiện `minConfidence` và `minSupport` và trả về kết quả này.

In [35]:
def association_rules(df, pattern_df, minSupport, minConfidence):
    # Step 1
    new_pattern_df = pattern_df.withColumn('count', f.size(pattern_df['items']))
    new_pattern_df = new_pattern_df.where(new_pattern_df['count'] > 1)
 
    # Step 2
    ls = []
    for row in new_pattern_df.collect():
        for permutation in list(permutations(row[0])):
            for split in range(1, len(permutation)):
                ls.append((list(permutation[:split]), list(permutation[split:]), row[1]))
    new_df = spark.createDataFrame(ls, ['antecedent', 'consequent', 'frequency'])
    
    # Step 3
    size = df.count()
    antecedent_df = pattern_df.alias('antecedent_df')
    consequent_df = pattern_df.alias('consequent_df')
    return new_df.join(antecedent_df, new_df['antecedent'] == antecedent_df['items'], 'inner')\
                 .withColumnRenamed('freq', 'antecedent_frequency')\
                 .join(consequent_df, new_df['consequent'] == consequent_df['items'], 'inner')\
                 .withColumnRenamed('freq', 'consequent_frequency')\
                 .withColumn('confidence', col('frequency') / col('antecedent_frequency'))\
                 .withColumn('lift', (col('frequency') * size) / (col('antecedent_frequency') * col('consequent_frequency')))\
                 .withColumn('support', col('frequency') / size)\
                 .select('antecedent', 'consequent', 'confidence', 'lift', 'support')\
                 .where((col('confidence') >= minConfidence) & (col('support') >= minSupport))

Chạy thử với DataFrame `test_df` ví dụ để tìm số lượng mẫu phổ biến và số lượng luật kết hợp để thấy sự khác biệt khi sử dụng thư viện MLlib FBGrowth và khi sử dụng hàm *association_rules()*.

In [36]:
# MLlib library
test_df = spark.createDataFrame([(0, [1, 2, 5]), (1, [1, 2, 3, 5]), (2, [1, 2])], ['id', 'items'])
fpGrowth = FPGrowth(itemsCol = 'items', minSupport = 0.5, minConfidence = 0.6)

model = fpGrowth.fit(test_df)
pattern_df = model.freqItemsets
rule_df = model.associationRules

pattern_df.show(5)
rule_df.show(5)
print('Số lượng mẫu phổ biến: ', pattern_df.count())
print('Số lượng luật kết hợp: ', rule_df.count())

+---------+----+
|    items|freq|
+---------+----+
|      [5]|   2|
|   [5, 1]|   2|
|[5, 1, 2]|   2|
|   [5, 2]|   2|
|      [2]|   3|
+---------+----+
only showing top 5 rows

+----------+----------+------------------+----+------------------+
|antecedent|consequent|        confidence|lift|           support|
+----------+----------+------------------+----+------------------+
|       [5]|       [1]|               1.0| 1.0|0.6666666666666666|
|       [5]|       [2]|               1.0| 1.0|0.6666666666666666|
|    [1, 2]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
|    [5, 2]|       [1]|               1.0| 1.0|0.6666666666666666|
|    [5, 1]|       [2]|               1.0| 1.0|0.6666666666666666|
+----------+----------+------------------+----+------------------+
only showing top 5 rows

Số lượng mẫu phổ biến:  7
Số lượng luật kết hợp:  9


In [38]:
test_df.show()

+---+------------+
| id|       items|
+---+------------+
|  0|   [1, 2, 5]|
|  1|[1, 2, 3, 5]|
|  2|      [1, 2]|
+---+------------+



In [37]:
# association_rules() function
new_rule_df = association_rules(test_df, pattern_df, minSupport = 0.5, minConfidence = 0.6)
pattern_df.show(5)
new_rule_df.show(5)
print('Số lượng mẫu phổ biến: ', pattern_df.count())
print('Số lượng luật kết hợp: ', new_rule_df.count())

+---------+----+
|    items|freq|
+---------+----+
|      [5]|   2|
|   [5, 1]|   2|
|[5, 1, 2]|   2|
|   [5, 2]|   2|
|      [2]|   3|
+---------+----+
only showing top 5 rows

+----------+----------+------------------+----+------------------+
|antecedent|consequent|        confidence|lift|           support|
+----------+----------+------------------+----+------------------+
|       [2]|    [5, 1]|0.6666666666666666| 1.0|0.6666666666666666|
|       [2]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
|    [1, 2]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
|       [1]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
|       [1]|    [5, 2]|0.6666666666666666| 1.0|0.6666666666666666|
+----------+----------+------------------+----+------------------+
only showing top 5 rows

Số lượng mẫu phổ biến:  7
Số lượng luật kết hợp:  12


**Nhận xét:** Sau khi khắc phục vấn đề này, số luật kết hợp của thuật giải sẽ lớn hơn số luật kết hợp của MLlib.

Áp dụng hàm *association_rules()* vào bài toán của chúng ta

In [45]:
pattern_df = pattern_df_2
new_rule_df = association_rules(df, pattern_df, minSupport = 0.005, minConfidence = 0.4)

pattern_df.show(5, False)
new_rule_df.show(5, False)
print('Số lượng mẫu phổ biến: ', pattern_df.count())
print('Số lượng luật kết hợp: ', new_rule_df.count())

+----------------------------------+----+
|items                             |freq|
+----------------------------------+----+
|[Total 0% Nonfat Greek Yogurt]    |993 |
|[Organic Large Green Asparagus]   |730 |
|[Green Onions]                    |1445|
|[Organic Tomato Basil Pasta Sauce]|772 |
|[Organic Egg Whites]              |834 |
+----------------------------------+----+
only showing top 5 rows

+--------------------------------------------+------------------------+------------------+-----------------+--------------------+
|antecedent                                  |consequent              |confidence        |lift             |support             |
+--------------------------------------------+------------------------+------------------+-----------------+--------------------+
|[Organic Hass Avocado, Organic Strawberries]|[Bag of Organic Bananas]|0.4613385315139701|3.910320890272384|0.005411214169759696|
+--------------------------------------------+------------------------+------

**Kết luận:** căn cứ trên dữ liệu giao dịch ban đầu `df` & kết quả mô hình FPGrowth với 2 ngưỡng support & confidence cao nhất có thể (thực nghiệm 2) thì suy được luật:

> Nếu khách hàng mua **[Organic Hass Avocado, Organic Strawberries]** thì xác suất mua **Bag of Organic Bananas** là 46.13%. Ngoài ra, số giao dịch mà khách hàng mua cả 3 món chiếm 0.5% trên tổng số giao dịch

